In [1]:
import os
import sys

import dotenv
import MetaTrader5 as mt5

try:
    dotenv.load_dotenv(".env")
except Exception as e:
    raise e
module_path = "../"
sys.path.append(module_path)

tgt_symbol = "USDJPY"

In [2]:
mt5.initialize()

authorized = mt5.login(
    int(os.environ["mt5_id"]),
    password=os.environ["mt5_password"],
    server=os.environ["mt5_server"],
)
authorized

True

In [4]:
results = []

In [5]:
rates = mt5.copy_rates_from_pos(tgt_symbol, mt5.TIMEFRAME_M1, 0, 1)
rates

array([(1749819120, 144.06, 144.081, 144.052, 144.078, 54, 3, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])

In [36]:
to_buy = False

if to_buy:
    order_type = mt5.ORDER_TYPE_BUY
else:
    order_type = mt5.ORDER_TYPE_SELL

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": tgt_symbol,
    "volume": 0.1,
    "price": rates[0][4],
    "deviation": 10,
    "magic": 100000000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type": order_type,
    "type_filling": mt5.ORDER_FILLING_IOC,  # depends on broker
}

result = mt5.order_send(request)
if result is None:
    raise Exception("failed order")
else:
    results.append(result)

In [44]:
mt5.positions_get()

(TradePosition(ticket=27599670, time=1749826541, time_msc=1749826541953, time_update=1749826541, time_update_msc=1749826541953, type=1, magic=100000000, identifier=27599670, reason=3, volume=0.1, price_open=144.214, sl=0.0, tp=0.0, price_current=144.167, swap=0.0, profit=470.0, symbol='USDJPY', comment='python script op', external_id=''),)

In [46]:
rates = mt5.copy_rates_from_pos(tgt_symbol, mt5.TIMEFRAME_M1, 0, 1)
if to_buy:
    sign = +1
else:
    sign = -1

request = {
    "action": mt5.TRADE_ACTION_SLTP,
    "tp": rates[0][4] + sign*0.2,
    "sl": rates[0][4] - sign*0.2,
    "comment": "python script modify",
    "position": results[-1].order
}

result = mt5.order_send(request)
if result is None:
    raise Exception("failed order")
result

OrderSendResult(retcode=10009, deal=0, order=0, volume=0.1, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=2367239113, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=144.34199999999998, tp=143.942, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='python script modify', position=27599670, position_by=0))

In [13]:
results[0]

OrderSendResult(retcode=10009, deal=0, order=27594167, volume=0.1, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=2367239098, retcode_external=0, request=TradeRequest(action=1, magic=100000000, order=0, symbol='USDJPY', volume=0.1, price=144.078, stoplimit=0.0, sl=0.0, tp=0.0, deviation=10, type=1, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))

In [26]:
results[-1]

OrderSendResult(retcode=10019, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='No money', request_id=3871166347, retcode_external=0, request=TradeRequest(action=1, magic=100000000, order=0, symbol='USDJPY', volume=0.1, price=145.112, stoplimit=0.0, sl=0.0, tp=0.0, deviation=10, type=1, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))

In [66]:
mt5.positions_get()

(TradePosition(ticket=27487172, time=1749652945, time_msc=1749652945927, time_update=1749652945, time_update_msc=1749652945927, type=0, magic=100000000, identifier=27487172, reason=3, volume=0.1, price_open=145.21, sl=0.0, tp=0.0, price_current=145.21, swap=0.0, profit=0.0, symbol='USDJPY', comment='python script op', external_id=''),)

In [ ]:
rates = mt5.copy_rates_from_pos(tgt_symbol, mt5.TIMEFRAME_M1, 0, 1)
if to_buy:
    order_type = mt5.ORDER_TYPE_SELL
else:
    order_type = mt5.ORDER_TYPE_BUY
    
for result in results:
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": tgt_symbol,
        "volume": 0.1,
        "price": rates[0][4],
        "deviation": 10,
        "magic": 100000000,
        "comment": "python script close",
        "type_time": mt5.ORDER_TIME_GTC,
        "type": order_type,
        "type_filling": mt5.ORDER_FILLING_IOC,  # depends on broker
        "position": result.order
    }

    closed_result = mt5.order_send(request)
    print(closed_result)

In [18]:
rates = mt5.copy_rates_from_pos(tgt_symbol, mt5.TIMEFRAME_M1, 0, 1)
request = {
    "action": mt5.TRADE_ACTION_PENDING,
    "symbol": tgt_symbol,
    "volume": 0.1,
    "price": rates[0][4] - 0.1,
    "deviation": 10,
    "magic": 100000000,
    "comment": "python script pending",
    "type_time": mt5.ORDER_TIME_GTC,
    "type": mt5.ORDER_TYPE_BUY_LIMIT,
    "type_filling": mt5.ORDER_FILLING_IOC,
    "expiration": 1000*60
}

result = mt5.order_send(request)
if result is None:
    raise Exception("failed order")

In [19]:
result

OrderSendResult(retcode=10009, deal=0, order=27599125, volume=0.1, price=0.0, bid=144.189, ask=144.189, comment='Request executed', request_id=2367239106, retcode_external=0, request=TradeRequest(action=5, magic=100000000, order=0, symbol='USDJPY', volume=0.1, price=144.189, stoplimit=0.0, sl=0.0, tp=0.0, deviation=10, type=2, type_filling=1, type_time=0, expiration=60000, comment='python script pending', position=0, position_by=0))

In [30]:
orders = mt5.orders_get()
orders

(TradeOrder(ticket=27599125, time_setup=1749825479, time_setup_msc=1749825479314, time_done=0, time_done_msc=0, time_expiration=0, type=2, type_time=0, type_filling=2, state=1, magic=100000000, position_id=0, position_by_id=0, reason=3, volume_initial=0.1, volume_current=0.1, price_open=144.059, sl=0.0, tp=0.0, price_current=144.272, price_stoplimit=0.0, symbol='USDJPY', comment='python script pe', external_id=''),)

In [35]:
rates = mt5.copy_rates_from_pos(tgt_symbol, mt5.TIMEFRAME_M1, 0, 1)
request = {
    "action": mt5.TRADE_ACTION_MODIFY,
    "price": rates[0][4] - 0.2,
    "tp": rates[0][4] + 0.2,
    "order": orders[0].ticket
}

result = mt5.order_send(request)
if result is None:
    raise Exception("failed order")
result

OrderSendResult(retcode=10009, deal=0, order=27599125, volume=0.1, price=144.058, bid=144.058, ask=144.058, comment='Request executed', request_id=2367239110, retcode_external=0, request=TradeRequest(action=7, magic=0, order=27599125, symbol='', volume=0.0, price=144.05800000000002, stoplimit=0.0, sl=0.0, tp=144.458, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

In [55]:
result

OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid request', request_id=0, retcode_external=0, request=TradeRequest(action=7, magic=100000000, order=27486434, symbol='USDJPY', volume=0.1, price=145.112, stoplimit=0.0, sl=0.0, tp=0.0, deviation=10, type=2, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))

In [63]:
request = {"action": mt5.TRADE_ACTION_REMOVE, "order": 27486601}
result = mt5.order_send(request)
result

OrderSendResult(retcode=10009, deal=0, order=27486601, volume=0.1, price=145.101, bid=145.101, ask=145.101, comment='Request executed', request_id=3871166366, retcode_external=0, request=TradeRequest(action=8, magic=0, order=27486601, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))